# Machine Learning, Assignment - 9 
## <div class="alert alert-info"><b>Comparing Naive Bayes classifier and Logistic Regression classifier  on image classification of cats and dogs.</b></div>

<div class="alert alert-info"> 1. Download a dataset from https://www.kaggle.com/c/dogs-vs-cats/data .  (2 points) </div>

In [1]:
# copy the file path into lists

import os

categories = ['Cat','Dog']

path = "C:\Seetha\data\kagglecatsanddogs_3367a\PetImages\\"

fileNames = []
category = []
for c in categories:           
     new_path = path + c      
     for file in os.listdir(new_path):   
        fileNames.append(os.path.join(new_path,file)) 
        category.append(c)

print(len(fileNames))
print(len(category))

25002
25002


In [2]:
# load the images 

import matplotlib.pyplot as plt
from PIL import Image
%matplotlib inline

import numpy as np

images = [] 
labels = []
for file in fileNames[:400]:    
    img = Image.open(file).convert('LA')
    img = img.resize((28, 28))
    img = np.ravel(img)
    images.append(img)
    labels.append(0)
    
print(len(images))

for file in fileNames[24600:25000]:    
    img = Image.open(file).convert('LA')
    img = img.resize((28, 28))
    img = np.ravel(img)
    images.append(img)
    labels.append(1)
    
print(len(images))

400
800


In [3]:
# Convert the images to a dataframe
import pandas as pd

df = pd.DataFrame(images)

print(df.shape)
df.head()

(800, 1568)


,0,1,2,3,4,5,6,7,8,9,...,1558,1559,1560,1561,1562,1563,1564,1565,1566,1567
0,173,255,187,255,197,255,204,255,209,255,...,28,255,8,255,4,255,2,255,2,255
1,35,255,41,255,45,255,45,255,43,255,...,59,255,91,255,57,255,57,255,39,255
2,57,255,54,255,61,255,58,255,53,255,...,70,255,73,255,140,255,143,255,172,255
3,220,255,242,255,223,255,230,255,232,255,...,237,255,223,255,206,255,207,255,211,255
4,116,255,80,255,192,255,205,255,242,255,...,86,255,89,255,80,255,70,255,76,255


In [4]:
# factorise the result as 0,1,2 and append to the dataframe
'''
labels, uniques = pd.factorize(category)

print(len(labels[:100]))
print(len(labels[24900:25000]))
print(uniques)
newlabels = []
newlabels.append(labels[:100])
newlabels.append(labels[24900:25000])
print(len(newlabels))'''

df.loc[:, 'Category'] = labels
df.head()

,0,1,2,3,4,5,6,7,8,9,...,1559,1560,1561,1562,1563,1564,1565,1566,1567,Category
0,173,255,187,255,197,255,204,255,209,255,...,255,8,255,4,255,2,255,2,255,0
1,35,255,41,255,45,255,45,255,43,255,...,255,91,255,57,255,57,255,39,255,0
2,57,255,54,255,61,255,58,255,53,255,...,255,73,255,140,255,143,255,172,255,0
3,220,255,242,255,223,255,230,255,232,255,...,255,223,255,206,255,207,255,211,255,0
4,116,255,80,255,192,255,205,255,242,255,...,255,89,255,80,255,70,255,76,255,0


In [5]:
df.tail()

,0,1,2,3,4,5,6,7,8,9,...,1559,1560,1561,1562,1563,1564,1565,1566,1567,Category
795,127,255,128,255,129,255,115,255,153,255,...,255,65,255,51,255,45,255,22,255,1
796,142,255,146,255,148,255,151,255,153,255,...,255,163,255,164,255,166,255,168,255,1
797,91,255,125,255,108,255,81,255,76,255,...,255,90,255,80,255,70,255,76,255,1
798,29,255,48,255,24,255,68,255,129,255,...,255,143,255,117,255,139,255,134,255,1
799,247,255,196,255,142,255,129,255,84,255,...,255,88,255,97,255,85,255,113,255,1


<div class="alert alert-info"> 2. Randomly sample the images to create training sets consisting of 70% of the  images.  Rest 30%  images form the  testing images. (1 point) </div>

In [6]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(df.iloc[:,:-1], df.Category, test_size=0.3,
                                                    stratify = df.Category, random_state=0)

print(X_train.shape, X_test.shape)
print(y_train.shape, y_test.shape)

(560, 1568) (240, 1568)
(560,) (240,)


<div class="alert alert-info"> 3A. Create Intensity and SVD based features </div>

intensity based features are using the X_train and Y_test as such.

In [7]:
from sklearn import decomposition

pca = decomposition.PCA(n_components=100, whiten=True)
pca.fit(X_train)

X_train_pca = pca.transform(X_train)
X_test_pca = pca.transform(X_test)

print(X_train_pca.shape)
print(X_test_pca.shape)

(560, 100)
(240, 100)


In [8]:
from sklearn import decomposition

svd = decomposition.TruncatedSVD(n_components=100)
svd.fit(X_train)

X_train_svd = svd.transform(X_train)
X_test_svd = svd.transform(X_test)

print(X_train_svd.shape)
print(X_test_svd.shape)

(560, 100)
(240, 100)


<div class="alert alert-info"> 3B. Naive Bayes Classifier </div>

In [9]:
from sklearn.naive_bayes import GaussianNB

GNB = GaussianNB()
GNB.fit(X_train, y_train)
print(GNB)

y_predict = GNB.predict(X_test) 

GaussianNB(priors=None, var_smoothing=1e-09)


In [10]:
GNBpca = GaussianNB()
GNBpca.fit(X_train_pca, y_train)
print(GNBpca)

y_predict_pca = GNBpca.predict(X_test_pca) 

GaussianNB(priors=None, var_smoothing=1e-09)


In [11]:
GNBsvd =  GaussianNB()
GNBsvd.fit(X_train_svd, y_train)
print(GNBsvd)

y_predict_svd = GNBsvd.predict(X_test_svd) 

GaussianNB(priors=None, var_smoothing=1e-09)


<div class="alert alert-info"> 4. Logistic Regression Classifier </div>

In [12]:
from sklearn.linear_model import LogisticRegression

LR = LogisticRegression(solver='liblinear', multi_class = 'auto') # default classfier

LR.fit(X_train, y_train)

y_prob  = LR.predict(X_test)

In [13]:
LRpca = LogisticRegression(solver='liblinear', multi_class = 'auto') # default classfier

LRpca.fit(X_train_pca, y_train)

y_prob_pca  = LRpca.predict(X_test_pca)

In [14]:
LRsvd = LogisticRegression(solver='liblinear', multi_class = 'auto') # default classfier

LRsvd.fit(X_train_svd, y_train)

y_prob_svd  = LRsvd.predict(X_test_svd)

<div class="alert alert-info"> 5A. Accuracy </div>

In [15]:
from sklearn.metrics import accuracy_score

accuracy = pd.DataFrame(columns=['Classifier', 'Features', 'Accuracy'])

accuracy.loc[0] = ['Gaussian NB', 'Intensity', accuracy_score(y_test, y_predict)*100]

accuracy.loc[1] = ['Gaussian NB', 'PCA', accuracy_score(y_test, y_predict_pca)*100]

accuracy.loc[2] = ['Gaussian NB', 'SVD', accuracy_score(y_test, y_predict_svd)*100]

accuracy.loc[3] = ['Logistic Reg', 'Intensity', accuracy_score(y_test, y_prob)*100]

accuracy.loc[4] = ['Logistic Reg', 'PCA', accuracy_score(y_test, y_prob_pca)*100]

accuracy.loc[5] = ['Logistic Reg', 'SVD', accuracy_score(y_test, y_prob_svd)*100]

print(accuracy)

     Classifier   Features   Accuracy
0   Gaussian NB  Intensity  54.166667
1   Gaussian NB        PCA  55.000000
2   Gaussian NB        SVD  54.166667
3  Logistic Reg  Intensity  50.000000
4  Logistic Reg        PCA  51.666667
5  Logistic Reg        SVD  50.000000


<div class="alert alert-info"> 5B. Confusion Matrix </div>

In [16]:
from sklearn.metrics import confusion_matrix 

conf_matrix = confusion_matrix(y_test, y_predict) 
print('Confusion Matrix Gaussian NB Intensity:', )
print(conf_matrix)

conf_matrix = confusion_matrix(y_test, y_predict_pca) 
print('Confusion Matrix Gaussian NB PCA:', )
print(conf_matrix)

conf_matrix = confusion_matrix(y_test, y_predict_svd) 
print('Confusion Matrix Gaussian NB SVD:', )
print(conf_matrix)

conf_matrix = confusion_matrix(y_test, y_prob) 
print('Confusion Matrix Logistic Regression Intensity:', )
print(conf_matrix)

conf_matrix = confusion_matrix(y_test, y_prob_pca) 
print('Confusion Matrix Logistic Regression PCA:', )
print(conf_matrix)

conf_matrix = confusion_matrix(y_test, y_prob_svd) 
print('Confusion Matrix Logistic Regression SVD:', )
print(conf_matrix)

Confusion Matrix Gaussian NB Intensity:
[[60 60]
 [50 70]]
Confusion Matrix Gaussian NB PCA:
[[98 22]
 [86 34]]
Confusion Matrix Gaussian NB SVD:
[[94 26]
 [84 36]]
Confusion Matrix Logistic Regression Intensity:
[[56 64]
 [56 64]]
Confusion Matrix Logistic Regression PCA:
[[59 61]
 [55 65]]
Confusion Matrix Logistic Regression SVD:
[[58 62]
 [58 62]]


<div class="alert alert-info"> 6. Compare the performance of the  above classifiers and report your observations. (2 points) </div>

From the reported accuracies,

Gaussian Naive Bayes Classifier gives better accuracy with intensity based features.

Logistic Regression gives better accuracy when PCa features are used.
